<a href="https://colab.research.google.com/github/muyeblog/implementAlgorithmFromScratch/blob/master/xgb_gbdt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GBDT 以及其他类型的提升树模型都是基于前向分布算法的(Forward stagewise algorithm)。而前向分布算法可以这样来理解，假设我们要使用决策树来预测一个人的年龄，刚开始的时候模型初始化会直接给出一个预测值$f_0(x)$，注意这个预测值不需要训练决策树来得到，而且不一定精确(比如刚开始模型初始预测为 0 岁，或者根据人群年龄分布给出一个相对合理的值)。接着在模型上一步所给出的预测基础上来训练第一棵决策树，此时模型的输出便是模型初始化的预测值加上第一棵决策树的输出值，然后我们继续添加第二课决策树，使得第二棵决策树能够在前面所构造的模型基础之上，让损失最小，不断的进行这一过程知道构建的决策树棵树满足要求或者损失小于一个阈值。当进行到第$m$步时，模型可以表示为
<center>

$f_m(x) = f_{m_1}(x) + \beta_mT(x;\Theta_{m})$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(1)

</center>

其中$f_{m-1}(x)$是已经构造的$m-1$棵决策树所组成的模型(包括模型初始值)，而$T(x;\Theta_{m})$是我们当前需要构造的第$m$棵决策树，$\beta_{m}$代表决策树的系数。需要提及一点的是，前向分布算法并不是拘泥于某一个分类器，此处仅以决策树为例。因为前面的$m-1$棵决策树已经训练好了，参数已经固定了，当系数$\beta_{m}$取一个固定值时，那么在第$m$步时，我们仅需要训练第$m$棵决策树的参数$\Theta_{m}$来最小化当前的损失：

<center>

$\hat{\Theta}_{m} = arg \min _{\Theta_m} \sum_{i=1}^N L(y_i,f_{m-1}(x_i) + \beta_{m} T(x_i;\Theta_{m}))$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(2)
</center> 

其中$N$代表样本的总个数，$L$代表损失函数。在前向分布算法搞清楚之后，再来看下机器学习如何最小化损失函数。

<center> 


假设现有一损失函数$J(\theta)$，当我们需要对这个参数为$\theta$的损失函数求最小值时，只要按照损失函数的负梯度方向调整参数$\theta$即可，因为梯度方向是使得函数函数增长最快的方向，沿着梯度的反方向也就是负梯度方向会使得函数减小最快，当学习率为$\rho$时，$\theta$的更新方法如下：
$\theta_i := \theta_{i} - \rho \frac{\partial J}{\partial \theta_i}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(3)
</center> 

那么同理，前向分布算法模型的损失$L$ 只和构造的模型$f_{m-1}(x)$ 有关，为了使得总体损失$J = \sum_{i} L(y_i, f_{m-1}(x_i))$进一步降低，需要对$f_{m-1}(x)$求导，而$f_{m-1}(x)$是针对$N$个样本的，所以对每个样本预测值求偏导数：

<center>

$f_m(x_i) := f_{m-1}(x_i) - \rho\frac{\partial J}{\partial f_{m-1}(x_i)}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(4)
</center>

这里的$\rho$和刚才提到的$\beta_m$的作用是相同的，我们在此令$\rho = 1$。因此，对于第$m$棵决策树而言，其拟合的不再是原有数据$(x_i,y_i)$的目标值$y_i$，而是负梯度，这样才能使得损失函数下降最快：

<center>

$\{(x_1,-\frac{\partial J}{\partial{f_{m-1}(x_1)}}),(x_2,-\frac{\partial J}{\partial{f_{m-1}(x_)}}),...,(x_n,-\frac{\partial J}{\partial{f_{m-1}(x_n)}})\}, m=1,2,...,M$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5)
</center>

到此为止，GBDT 中梯度提升的含义理清了，也就是说，模型开始时会给出一个初始预测值，在构建后续的决策树时，每个样本的目标值就变成了损失函数对当前模型在这个样本产生的负梯度，这样做的目的是为了最小化损失函数，进而提升模型性能。最后将所有的决策树组合起来就是 GBDT 模型。



## 2 分类与回归树

在上面，弄清楚了Gradient Boosting的含义，就是通过拟合梯度来提升模型性能，那么 Decision Tree 在 GBDT 中的具体形式是什么呢？本小节来介绍 GBDT 所使用的分类与回归树 (Classification and regression tree,CART)。

从名字上看，CART 决策树可以处理分类和回归问题，这当然取决于分裂准则的选择。当采用平方误差损失时，可以用于回归问题，当采用基尼系数知道分裂时，可以用于分类问题。但在 GBDT 采用 CART 决策树作为基函数时，尽管 GBDT 可以处理分类与回归问题(实际上这也取决于 GBDT 损失函数的选择)，CART 决策树的分裂准则一直采用平方误差损失(比如在sklearn的实现中，均默认采用"friedman_mse"，一种基于平方误差损失的改进版)，因此在这里，我们主要介绍一下当损失函数为平方误差时 CART 决策树的生成流程。

-------------------算法流程图分割线----------------------

#### 算法 1： CART 生成流程

给定训练集$D$
<br>
(1) 依次遍历每个变量的取值，寻找最优切分变量$j$与切分点$s$，进而求解：

<center>


$\min_{j,s}[\min_{c_1}\sum_{x_i \in R_1(j,s)} (y_i - c_1)^2 + \min_{c_2}\sum_{x_i\in R_2(j,s)}(y_i - c_2)^2]$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(6)
</center>

这一步就是对当前点寻找最优切分点，使得切分之后两个节点总体平方误差最小。
<br>
(2) 接着用选定的对$(j,s)$划分区域并决定相应的输出值：
<center>

$R_1(j,s) = \{x | x^{(j)} \leq s\}, R_{2}(j,s) = \{x|x^{(j)} > s\},$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(7)
</center>
<center>


$\hat{c}_m = \frac{1}{N_m}\sum_{x_i\in R_{m}(j,s)} y_i,x\in R_m,m=1,2.$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(8)
</center>

$R_1(j,s),R_2(j,s)$分别代表的是左子节点和右子节点，而对两个节点上的估计值$\hat{c}_m$采用相应子节点上目标值 的均值来表示(**注意这里划分区间时是一直按照平方误差损失最小划分的，但$\hat{c}_m$采用均值**)

<br>
(3)继续对两个子区域调用步骤(1),(2)，直到满足停止条件(比如子节点上样本个数过少或者决策树已经达到指定的深度)

<br>
(4) 将输入空间划分为$M$个区域$R_1,R_2,...,R_M$，生成决策树：
<center>

$f(x) = \sum_{m=1}^M \hat{c}_mI(x\in R_m)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(9)
</center>

$f(x)$便是我们学习到的 CART 决策树，$I(x\in R_m)$表示相应样本属于的区域，在相应区域其值为 1，否则为 0.


---------------------------------算法流程图分割线-----------------------------------



## 处理回归问题的 GBDT

在介绍了 GBDT 中 Gradient Boosting 和 Decision Tree 的含义之后，这里会给出处理回归问题的 GBDT 形式。 对于回归问题来说，我们以平方误差损失函数为例，介绍 GBDT 的处理方法。

<center>


$L(y,f(x)) =  \frac{1}{2}\cdot (y-f(x))^2,$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(10)
</center>

当损失函数如公式(10)所示时，对于总体的损失函数 $J = \sum_{i}L(y_i,f_{m-1}(x_i))$，其对每个样本当前预测值产生的梯度为：
<center>

$\frac{\partial{J}}{\partial{f_{m-1}(x_i)}} = \frac{\partial{\sum_{i}L(y_i,f_{m-1}(x_i))}}{\partial{f_{m-1}(x_i)}} = \frac{\partial{L(y_i,f_{m-1}(x_i))}}{\partial{f_{m-1}(x_i)}} = f_{m-1}(x_i) - y_i,$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(11)
</center>

那么对应的负梯度：

<center>

$-\frac{\partial{J}}{\partial{f_{m-1}(x_i)}} = y_i - f_{m-1}x_i ,$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(12)
</center>

其中$y_i - f_{m-1}(x_i)$便是当前模型对数据需要拟合的残差(Residual)，因此对于回归问题的提升树模型而言，每次所添加的基函数只需要去拟合上次遗留的残差即可。

所以回归问题的 GBDT 方法流程可以写为：

---------------------------------算法流程图分割线-----------------------------------


#### 算法 2：回归问题的 GBDT 流程
输入：训练数据集 $T = \{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\},x_i\in \chi \subseteq R^n,y_i \in Y\subseteq R$，以及基函数系数或者学习率$\beta_m$。

输出：提升树$f_M(x)$。

(1)  初始化$f_0(x)=\frac{1}{N}\sum_{i}^N y_i$（为什么这样初始化可以下面参考第6小节第1条）

(2) 对 $m=1,2,...,M$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a) 计算残差：
<center>

$r_{mi} = y_i -f_{m-1}(x_i),i=1,2,...N$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(13)
</center>

> m表示第 m 棵树，即第 m 个弱学习器；i表示第 i 个样本。

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b) 拟合残差$r_{mi}$学习一个回归树，得到第$m$棵树的叶结点区域$R_{mj},j=1,2,...,J$.**注意这一步是按照平方误差损失进行分裂节点的**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(c) 对 $j=1,2,...,J$，计算
<center>

$c_{mj} = arg\min_{c} \sum_{x_i\in R_{mj}} L(y_i,f_{m-1}(x_i) + c)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(14)
</center>

需要注意的是，这一步是为了计算怎对叶结点区域内的样本进行赋值，使得模型损失最小，因此这一步是需要根据当前模型的损失函数来计算的，当前模型是针对回归问题的模型，我们的损失函数是平方误差损失，所以这一步的计算结果为（计算过程放在了下面第6小节第2条）：
<center>


$c_{mj} = \frac{1}{N_{mj}}\sum_{x_i\in R_{mj}} r_{mi}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(15)
</center>

即每个叶结点区域残差的均值。

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(d) 更新$f_m(x) = f_{m-1}(x) + \beta_{m}\sum_{j=1}^J c_{mj}I(x\in R_{mj})$.

(3) 得到回归问题提升树
<center>

$f_M(x) = \sum_{m=1}^M\beta_m\sum_{j=1}^J c_{mj}I(x\in R_{mj})$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(16)
</center>


---------------------------------算法流程图分割线-----------------------------------


需要再提一遍的是，$\beta_m$可以设置为 1，但它的作用和梯度下降时的学习率一致，能够使得损失函数收敛到更小的值，因此可以调小$\beta_m$,而一个较小的学习率会使得模型趋向于训练更多的决策树，使得模型更加稳定，不至于过拟合。


### 4 处理二分类问题的 GBDT

因为处理回归问题的 GBDT 采用的是以平方误差为分裂准则的 CART 决策树，其叶结点的输出其实是整个实数范围。

可以参考逻辑回归在线性回归的基础上添加激活函数的方法，来使得 GBDT 同样能够应对分类问题。

参考之前的文章——[逻辑回归简介及实现](https://zhuanlan.zhihu.com/p/130209974)，我们可以把二分类GBDT的损失函数定义为

<center>

$L(y,f(x)) = -y\log\hat{y} - (1-y)\log{1-\hat{y}},$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(17)
</center>

<center>

$\hat{y} = \frac{1}{1+e^{-f(x)}},$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(18)
</center>

其中$\hat{y}$代表的是当前样本类别为 1 的概率$P(y=1|x)$。那么对于总体损失函数$J = \sum_{i}L(y_i,f_{m-1}(x))$，其对每个样本当前预测值产生的梯度为：

<center>

$\frac{\partial J}{\partial f_{m-1}(x_i)} = \frac{\partial{L(y_i),f_{m-1}(x_i)}}{\partial{f_{m-1}(x_i)}}$<br>
$=\frac{\partial{[-y_i\log\frac{1}{1+e^{-f_{m-1}(x_i)}} - (1-y_i)\log(1 - \frac{1}{1 + e^{-f_{m-1}(x_i)}})]}}{\partial{f_{m-1}(x_i)}}$<br>
$=\frac{\partial{[y_i\log(1 + e^{-f_{m-1}(x_i)}) + (1-y_i)[f_{m-1}(x_i) + \log(1 + e^{-f_{m-1}(x_i)})]]}}{\partial{f_{m-1}(x_i)}},$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(19)<br>
$=\frac{\partial{[(1 - y_i)f_{m-1}(x_i) + \log(1 + e^{-f_{m-1}(x_i)})]}}{\partial{f_{m-1}(x_i)}}$<br>
$=1 - y_i + \frac{-e^{-f_{m-1}(x_i)}}{1 + e^{-f_{m-1}(x_i)}}$<br>
$= \frac{1}{1 + e^{-f_{m-1}(x_i)}} - y_i$<br>
$=\hat{y}_i - y_i$
</center>

也就是当前模型预测的概率值与样本真实类别值之间的差值，那么负梯度可以写为：
<center>

$-\frac{\partial J}{\partial f_{m-1}(x_i)} = y_i - \hat{y}_i,$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(20)
</center>

因此对于 处理二分类问题的 GBDT 来说，其每次拟合的是真实样本值与模型预测值的差值，所以二分类问题的 GBDT 方法流程可以写为：

---------------------------------算法流程图分割线-----------------------------------

#### 算法 3： 二分类问题的 GBDT 流程
输入：训练数据集 $T = \{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\},x_i \in \chi \subseteq R^n,y_i\in\{0,1\}$，以及基函数系数或者学习率$\beta_m$。

输出：提升树$f_M(x)$

(1) 初始化$f_0(x) = \log \frac{p_1}{1-p_1},$其中$p_1$代表的是样本中$y=1$的比例（为什么这样初始化可以参考下面第6小节第3条）。<br>

(2) 对 $m=1,2,...,M$
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a) 计算负梯度：
<center>

$r_{mi} = y_i - \hat{y}_i,i=1,2,...,N$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(21)
</center>

其中$\hat{y}_i = \frac{1}{1 + e^{-f_{m-1}(x_i)}}$代表$y_1 = 1$的概率。

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b) 拟合负梯度$r_{mi}$学习一个回归树，得到第$m$棵树的叶结点区域$R_{mj},j=1,2,...,J$。注意这一步也是按照平方误差损失进行分裂节点的。

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(c) 对 $j = 1,2,...,J$，计算
<center>

$c_{mj} = arg\min_c \sum_{x_i\in R_{mj}} L(y_i,f_{m-1}(x_i) + c)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(22)
</center>

同样的，当前模型是针对二分类问题的模型，损失函数为交叉熵损失，所以这一步计算结果为（计算过程请见下面第6小节第4条）：


<center>

$c_{mj} = \frac{\sum_{x_i \in R_{mj}}r_{mi}}{\sum_{x_i \in R_{mj}} (y_i - r_{mi})(1 -  y_i + r_{mi} )}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(23)
</center>


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(d) 更新$f_m(x) = f_{m-1}(x) +\beta_{m}\sum_{j=1}^J c_{mj}I (x\in R_{mj})$。

(3) 得到二分类问题提升树 
<center>

$f_{M}(x) = \sum_{m=1}^M \beta{m}\sum_{j=1}^J  c_{mj}I(x\in  R_{mj})$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(24)
</center>


---------------------------------算法流程图分割线-----------------------------------

### 5 处理多分类问题的 GBDT

二分类问题对于 GBDT 来讲，采取了和逻辑回归同样的方式，即利用 sigmoid 函数将输出值归一化。而在处理多分类问题时，GBDT 可以采用 softmax 函数一步步求解。

需要说明的是，对于多分类问题每个样本的类标都要先经过 one-hot 编码，这样才能划分成多个二分类问题来求解。假设数据集$D$的类别树$K=4$，样本个数为$N$，并且样本$x_1,x_2,x_3,x_4$的类标分别为$y_1=0,y_2=1,y_3=2,y_4=3$，那么在处理多分类问题时，GBDT 将会通过 one-hot 编码将原有的一个多分类问题转化为 四个二分类问题，如下所示：
![](https://pic2.zhimg.com/80/v2-ee573ec7d14ef8b1fbe0a7e68f3c1699_1440w.jpg)


由于在表里添加了很多内容，所以 这里进行一些说明，

1. one-hot 编码的意思是每个样本的类别都会被转为一个$K$维向量，并且向量中仅有一个值为 1，比如$y_2=1$被转为$\{0,1,0,0\}$；

2. 在转换之后，$y_{i,j}$代表的是第$i$个样本在第$j$个二分类问题的表示值，比如$y_{3,2}=0$代表的是第 3 个样本$x_3$在第 2 个二分类问题中类标变为 0，其中$i=1,2,...,N$，$j=1,2,...,K$。

3. $F_0(x) = 0$ 是模型预测的初始值，后续会提到，还可以采用其他的初始化方法。

4. $F_1(x)$代表的是进行第 1 轮学习后的模型，由$F_0(x) + \{T_{1,1}(x,\theta),T_{2,1}(x,\theta), T_{3,1}(x,\theta),T_{4,1}(x,\theta)\}$共同组成的，同理$F_M(x)$代表的是经过$M$轮学习后的最终 GBDT 模型，由$F_{M-1}(x) + \{ T_{1,M}(x,\theta),T_{2,M}(x,\theta),T_{3,M}(x,\theta),T_{4,M}(x,\theta)\}$。

5. $T_{j,M}(x,\theta)$代表的是对第 $j$个二分类问题进行第$m$轮学习时，拟合负梯度所产生的决策树，其中$j=1,2,...,K, m=1,2,...,M$。比如$T_{4,2}(x,\theta)$代表的是针对第 4 个二分类问题，进行第 2 次学习之后产生的决策树，并且其训练样本$x_1,x_2,x_3,x_4$的类标为$\{0,0,0,1\}$，当然这是原始的类别，$T_{4,2}(x,\theta)$在学习的时候，样本类标已经变为基于上轮学习产生的负梯度了。

当进行了$m$轮学习之后，GBDT 模型 变为$F_{m}(x)$，那么对于样本$x$，其属于每个类别$j$的概率可以预测为
<center>

$P(y=j|x) = \frac{e^{F_{j,m}(x)}}{\sum_{l=1}^K e^{F_{l,m}(x)}},j=1,2,..,K$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(25)
</center>

其中$F_{j,m}(x)$代表的是模型经过$m$轮学习之后，在第$j$类二分类问题中，对当前样本$x$的预测值。

对于多分类问题的 GBDT 来说，在经过$m$轮后，模型$F_{m}(x)$对于第$i$个样本的损失函数为
<center>

$L(y_{i,*},F_{m}(x_i)) = -\sum_{j=1}^K y_{i,j}\log P(y_{i,j}|x_i) = -\sum_{j=1}^K y_{i,j}\log\frac{e^{F_{j,m}(x_i)}}{\sum_{l=1}^K e^{F_{l,m}(x_i)} }$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(26)
</center>

推导出损失函数的详细过程，参考下面第 6 节第 5 条。

还是老步骤，接下来求出单个样本损失函数的负梯度$\frac{\partial P(y_{i,j}|x_i)}{\partial{F_{k,m}(x_i)}}$:

<center>

$\frac{\partial{ P(y_{i,j} | x_i) }}{\partial{ F_{k,m}(x_i) }} = \frac{\partial{\frac{ e^{F_{j,m}(x_i)} }{ \sum_{l=1}^K e^{F_{l,m}(x_i) }  }}}{\partial{  F_{k,m}(x_i)  }}$<br>
$= \frac{ \frac{ \partial{ e^{F_{j,m}(x_i)} }}{ \partial{ F_{k,m}(x_i)}} \sum_{l=1}^K e^{F_{l,m}(x_i)} - e^{F_{j,m}(x_i)} \frac{ \partial{ \sum_{l=1}^K e^{F_{l,m}(x_i)} } }{ \partial{F_{k,m}(x_i)}  }}{ [\sum_{l=1}^K e^{F_{l,m}(x_i)} ]^2}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(27)
</center>

这里分为两种情况求解，第一种当 $j=k$时：
<center>

$\frac{\partial{ e^{F_{j,m}(x_i)} }}{\partial{ F_{k,m} (x_i)}} = e^{F_{j,m}(x_i)}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(28)
</center>

因而：
<center>

$\frac{
  \partial{ P(y_{i,j} | x_i ) } 
  } { \partial{ F_{k,m}(x_i) }
} = \frac{ e^{F_{j,m}(x_i)} \sum_{l=1}^K e^{ F_{l,m}(x_i)} - e^{F_{j,m}(x_i)}e^{ F_{k,m}(x_i)}}{[ \sum_{l=1}^K e^{F_{l,m}(x_i)} ]^2} $<br>
$ = P(y_{i,j} | x_i)\cdot [1 - P(y_{i,k} | x_i)]$.&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(29)
</center>

第二种情况，即当 $j \neq k$时:

<center>

$\frac{\partial{e^{F_{j,m}(x_i)}}}{\partial{F_{k,m}(x_i)}} = 0$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(30)
</center>

因而：
<center>

$\frac{\partial { P(y_{i,j}|x_i)}}{\partial{F_{k,m}(x_i)}} = \frac{ -e^{F_{j,m}(x_i)} e^{F_{k,m}(x_i)} }{ [ \sum_{l=1}^K e^{F_{l,m}(x_i)}]^2}$<br>
$=  - P(y_{i,j} |x_i)\cdot P(y_{i,k}|x_i)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(31)
</center>

回到我们要求的梯度：

<center>

$\frac{ \partial{ L(y_{i,*},F_{m}(x_i)) }}{ \partial{F_{k,m}(x_i)}} = \frac{\partial{-  \sum_{j=1}^K y_{i,j}\log P(y_{i,j}|x_i) }}{\partial{F_{k,m}(x_i) }}$<br>
$=-\sum_{j=1}^K y_{i,j}\frac{1}{P(y_{i,j}|x_{i})} \frac{\partial{P(y_{i,j}|x_i)}}{\partial{F_{k,m}(x_i)}}$<br>
$=-y_{i,k}\frac{1}{P(y_{i,k}|x_i)}P(y_{i,k}|x_i)\cdot [1 - P(y_{i,k}|x_i)] - \sum_{j\neq k}^K y_{i,j} \frac{1}{P(y_{i,j}|x_i)}[-P(y_{i,j}|x_i)\cdot P(y_{i,k}|x_i)]$<br>
$ = -y_{i,k} + y_{i,k}P(y_{i,k}|x_i) + \sum_{j\neq k}^K y_{i,j}P(y_{i,k}|x_i)$<br>
$ = -y_{i,k} + \sum_{j=1}^K y_{i,j}P(y_{i,k}|x_i)$<br>
$=-y_{i,k} + P(y_{i,k}|x_i)\sum_{j=1}^K y_{i,j}$<br>
$=P(y_{i,k}|x_i) - y_{i,k}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(32)
</center>

因此所求的单样本损失函数的负梯度：

<center>

$-\frac{\partial{ L(y_{i,*},F_{m}(x_i))  }}{\partial{F_{k,m}(x_i)}} = y_{i,k} - P(y_{i,k}|x_i),$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(33)
</center>




对于处理多分类问题的 GBDT 来说，其每次拟合的是真实类标与模型预测值之差，所以多分类 问题的 GBDT方法流程可以写为：

---------------------------------算法流程图分割线-----------------------------------


#### 算法 4 多分类问题的 GBDT 流程
输入：训练数据集$T = \{(x_1,y_1),(x_2,y_2),...,(x_N,y_N)\}, x_i\in \chi,\subseteq R^n, y_i\in \{0,1,...,K\}$，以及基函数系数或者学习率$\beta_m$。

输出：提升树$f_M(x)$。

(1) 初始化 $F_{k,0}(x) = 0$，即$P(y_{i,k}|x) = \frac{1}{K},k=1,2,...,K$。这样初始化是考虑到当我们对数据一无所知时，最好的假设就是每类发生的概率相等，但当我们知道数据集类别分布情况后，我们可以按照类别比例来设置初始化。

(2) 对$m = 1,2,...,M$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(a)对 $i=1,2,...,N,k=1,2,...,K,$计算$P(y_{i,k}|x_i) = \frac{e^{F_{k,m-1}(x_i)}}{\sum_{l=1}^K e^{F_{l,m-1}(x_i)}}$。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b) 对$k=1,2,...,K$:<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b1)计算负梯度：
<center>

$r_{i,k,m} = y_{i,k} - P(y_{i,k}|x_i),i=1,2,...,N$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(34)
</center>

其中$r_{i,k,m}$表示的是模型在第$m$轮学习中，第$k$个二分类问题里，对第$i$个样本产生的负梯度 。<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b2)拟合负梯度$r_{i,k,m}$学习一个回归树，得到第$m$棵树的叶结点区域$R_{j,k,m},j=1,2,...,J$。注意这一步也是按照平方误差损失进行分裂节点的。

<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b3)对$j=1,2,...,J$计算：
<center>

$c_{j,k,m} = arg\min_c \sum_{x_i\in R_{j,k,m}} L(y_{i,k},F_{k,m-1}(x_i) + c)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(35)
</center>

同样的，当前模型是针对多分类问题的模型，根据多分类模型的损失函数，这一步计算结果 为(这一步本文作者没有推导处理，感兴趣的同学可以看一下论文原文: [Greedy function approximation: a gradient boosting machine](https://link.zhihu.com/?target=https%3A//projecteuclid.org/download/pdf_1/euclid.aos/1013203451)):
<center>

$c_{j,k,m} = \frac{K-1}{K}\frac{ \sum_{x_i\in R_{j,k,m} r_{i,k,m}} }{\sum_{x_i\in R_{j,k,m}} |r_{i,k,m}|(1 - |r_{i,k,m}|)}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(36)
</center>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(b4) 更新$F_{k,m}(x) = F_{k,m-1}(x) + \beta_{m}\sum_{j=1}^J c_{j,k,m}I(x\in R_{j,k,m})$。

(3)得到多分类问题提升树：
<center>

$F_{k,M} (x) = \sum_{m=1}^M \beta_{m}\sum_{j=1}^J c_{j,k,m}I(x\in R_{j,k,m})$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(37)
</center>

---------------------------------算法流程图分割线-----------------------------------




### 6 公式推导

**(1) 对于回归问题的 GBDT 模型，为什么初始化为$f_0(x) = \frac{1}{N}\sum_{i}^N y_i$?**

由于回归问题的损失函数为平方差损失$J=\sum_{i}^NL(y_i,f(x_i)) = \sum_{i}^N\frac{1}{2}\cdot (y_i - f(x_i))^2$，为了使得损失函数最小化，那么我们在初始化的时候也希望能够有一个好的起点，即希望能够进行一个使得损失函数最小的初始化操作：
<center>

$\frac{\partial J}{\partial f_0(x)} = \frac{\partial \sum_{i}^N \frac{1}{2}\cdot (y_i -f_0(x))^2 }{\partial f_0(x)}$<br>
$= \sum_i^N (f_0(x) - y_i)$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(38)
</center>

令 $\frac{\partial J}{\partial f_0(x)}=0$可得，$f_0(x) = \frac{1}{N}\sum_{i}^N y_i$。

**(2) 对于回归问题的 GBDT，叶结点输出值$c_{mj}$的计算过程。**

由于$c_{mj} = arg\min_c \sum_{x_i\in R_{mj}} L(y_i, f_{m-1}(x_i) + c)$，那么为了使损失函数最小，我们令 $\frac{\partial L}{\partial c} = 0$即可，
<center>

$\frac{\partial L}{\partial c} = \frac{\partial \sum_{x_i\in R_{mj}} L(y_i,f_{m-1}(x_i)+c)}{\partial c}$<br>
$=\frac{\partial \sum_{x_i \in R_{mj}} \frac{1}{2} (y_i - f_{m-1}(x_i) -c)^2}{\partial c}$<br>
$ = \sum_{x_i \in R_{mj}}[c - (y_i - f_{m-1}(x_i))]$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(39)
</center>

由$\frac{\partial L}{\partial c}=0$得：
<center>

$c_{mj} = \frac{1}{N_{mj}} \sum_{x_i\in R_{mj}}(y_i  - f_{m-1}(x_i))$<br>
$=\frac{1}{N_{mj}} \sum_{x_i \in R_{mj}} r_{mi}$,&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(40)
</center>

其中 $N_{mj}$代表的是第$m$棵决策树的第$j$个叶结点区域包含的样本数量。


**(3) 对于二分类问题的 GBDT，为什么初始化为 $f_0(x) = \log\frac{p_1}{1-p_1}$ ? 其中$p_1$代表的是样本中$y=1$的比例。**

和 回归问题的 GBDT 初始化操作一样，这样初始化是为了最小化损失函数。对于二分类问题的 GBDT，其损失函数为：
<center>

$J= \sum_{i=1}^N L(y_i,f(x_i))$<br>
$ = \sum_{i=1}^N [-y_i\log\hat{y}_i - (1 - y_i)\log(1  - \hat{y}_1)]$<br>
$=\sum_{i=1}^N  [-y_i \log \frac{1}{1 + e^{-f(x_i)}} - (1-y_i)\log (1 - \frac{1}{1 + e^{-f(x_i)}})]$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(41)
</center>

那么初始化函数$f_0(x)$，我们将损失函数对其求导：
<center>

$\frac{\partial J}{\partial{f_0(x)}} = \frac{\partial \sum_{i=1}^N [-y_i \log \frac{1}{1 + e^{-f_0(x_i)}} - (1-y_i)\log(1 - \frac{1}{1 + e^{-f_0(x_i)}})]}{\partial f_0(x)}$<br>
$= \frac{\partial \sum_{i=1}^N [y_i\log(1 + e^{-f_0(x_i)}) -  (1-y_i)\log \frac{ e^{-f_0(x_i)} }{ 1 + e^{-f_0(x_i)} } ]}{\partial f_0(x)}$<br>
$= \frac{\partial \sum_{i=1}^N [y_i\log(1 + e^{-f_0(x_i)}) -  (1-y_i)(  -f_0(x_i) - \log( 1 + e^{-f_0(x_i)})) ]}{\partial f_0(x)} $ <br>
$= \frac{\partial \sum_{i=1}^N [(1-y_i)f_0(x_i) +  \log( 1 + e^{-f_0(x_i)})) ] }{\partial f_0(x)} $<br>
$ = \sum_{i=1}^N (1 - y_i - \frac{e^{-f_0(x_i)}}{ 1 + e^{-f_0(x_i)}})$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(42)
</center>

注意其中的$f_0(x_i)$对所有的$i=1,2,...,N$都是一个常数值，最后我们令$\frac{\partial J}{\partial f_0(x)} = 0$，在训练集中不全为负样本或全为正样本的情况下 ，$f_0(x) = \log\frac{\sum_{i=1}^N  y_i}{N - \sum_{i=1}^N  y_i}$,也就是$f_0(x) = \log\frac{p_1}{1 - p_1}$,$p_1$为训练集中正样本的比例。
(训练集中不全为负样本或全为正样本是因为 对数函数的变量定义域大于 0，即$\frac{p_1}{1-p_1}$大于 0)。


**(4) 对于二分类问题的 GBDT ，叶结点 $f_{mj} 输出值的计算过程$**

在进行第$m$轮学习时，损失函数为$J = \sum\limits_{x_i\in R_{mj}} L(y_i,f_{m-1}(x_i)+ c)$，由泰勒公式得：

$J = \sum\limits_{x_i\in R_{mj}} L(y_i,f_{m_1}(x_i) + c) \approx \sum\limits_{x_i\in R_{mj}}[ L(y_i,f_{m-1}(x_i))  + \frac{\partial{ L(y_i,f_{m-1}(x_i)) }}{\partial{ f_{m-1}(x_i)  }}\cdot c  + \frac{1}{2} \frac{\partial^2{ L(y_i,f_{m-1}(x_i)) }}{\partial^2{ f_{m-1}(x_i)  }}\cdot c^2],$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(43)

而
<center>

$\frac{\partial{ L(y_i,f_{m-1}(x_i)) }}{\partial{ f_{m-1}(x_i)  }} = \frac{1}{1 + e^{-f_{m-1}(x_i)}} - y_i$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(44)

$\frac{\partial^2{ L(y_i,f_{m-1}(x_i)) }}{\partial^2{ f_{m-1}(x_i)  }} = \frac{ e^{-f_{m-1}(x_i)} }{ [ 1 + e^{-f_{m-1}(x_i)}]^2}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(45)
</center>

那么损失函数可以写为：
<center>

$J\approx  \sum\limits_{x_i \in R_{mj}}[[-y_i\log \frac{1}{1 + e^{-f_{m-1}(x_i)}} - (1-y_i)\log(1 - \frac{1}{ e^{-f_{m-1}(x_i)} })] + [\frac{1}{ 1 + e^{-f_{m-1}(x_i)}} -y_i]\cdot c + \frac{1}{2}\frac{e^{-f_{m-1}(x_i)}}{[1 + e^{-f_{m-1}(x_i)}]^2}\cdot c^2   ]$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(46)
</center>

所以求导可得：
<center>

$\frac{\partial J}{\partial c} =\frac{\partial \sum\limits_{x_i\in R_{mj}}[[\frac{1}{1 +  e^{-f_{m-1}(x_i)}} - y_i]\cdot c + \frac{1}{2} \frac{e^{-f_{m-1}(x_i)}}{[1 + e^{-f_{m-1}(x_i)}]^2}\cdot c^2]}{\partial  c}$<br>
$=\sum\limits_{x_i\in R_{mj}} [\frac{1}{1 + e^{-f_{m-1}(x_i)}}-y_i +  \frac{e^{-f_{m-1}(x_i)}}{[1 + e^{-f_{m-1}(x_i)}]^2}\cdot c ]$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(47)

</center>

令$\frac{\partial J}{\partial c}=0$得：
<center>

$c_{mj}=\frac{\sum\limits_{x_i\in R_{mj}}r_{mi}}{\sum\limits_{x_i\in R_{mj}} (y_i - r_{mi})(1 - y_i + r_{mi})}$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(48)
</center>







**(5) 多分类问题的 GBDT, 经过$m$轮学习，第$i$个样本的损失函数推导过程。**

通过上面的回顾，可以知道逻辑回归的损失函数是通过极大似然估计法得到，逻辑回归中的条件概率为：

<center>

$P(Y|x) = P(Y=1|x)^yP(Y=0|x)^{1-y},$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(49)
</center>

那么对于上文表中的第一个样本：

![](https://pic1.zhimg.com/80/v2-2bdd3e58c541b4c1979251efc9559c4c_1440w.jpg)

来说，此时的条件概率可以写为：

<center>

$P(Y|x_1) = P(y_{1,1}=1|x_1)^{y_{1,1}}\cdot P(y_{1,2}=1|x_1)^{y_{1,2}}\cdot P(y_{1,3}=1|x_1)^{y_{1,3}}\cdot P(y_{1,4}=1|x_1)^{y_{1,4}}$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(50)
</center>

所以处理多分类问题的 GBDT 中单个样本的条件概率可以写为：

<center>

$P(Y|x_i)=\prod\limits_{j=1}^KP(y_{i,j}|x_i)^{y_{i,j}}$<br>
$=\prod\limits_{j=1}^K(\frac{e^{F_{j,m}(x)}}{\sum_{l=1}^K e^{F_{l,m}(x)}})^{y_{i,j}},j=1,2,...,K.$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(51)
</center>

对于单个样本来说，我们希望其条件概率最大化，但损失函数往往要求最小化，所以对条件概率求负对数即可得到经过$m$轮训练之后，模型对单个样本的损失函数：

<center>

$L(y_{i,*},F_{m}(x)) = -\sum\limits_{j=1}^K y_{i,j}\log P(y_{i,j}|x_i) = -\sum\limits_{j=1}^K y_{i,j}\log\frac{e^{F_{j,m}(x_i)}}{\sum_{l=1}^K e^{F_{l,m}(x_i)}}.$&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(52)
</center>

### 7. 总结

总的来说，这篇文章主要介绍了GBDT如何处理回归问题、二分类问题和多分类问题，但是还是有很多不足，比如多分类问题叶结点输出值的推导还不清楚，而且每类问题缺乏实例的解释。另外，我觉得掌握任何一种算法不光要做到能向其他人讲清楚原理，还要能够用代码实现它，但因为这个专栏的文章主要是学习CTR预估，后续还要学习很多的内容，因此代码实现在这里先留个坑，等我过一遍CTR预估经典方法之后再来完成。


>欢迎查看[我的github获取本文的pdf版本以及GBDT原始论文](https://link.zhihu.com/?target=https%3A//github.com/crazycharles/TheWayToCTR)^o^



### 1.为什么使用 abc?

Abstract base classes由一组接口组成，检查比hasattr()更严格。通过定义一个抽象基类，可以为一组子类定义一个通用的API。这对于第三方为应用提供插件等非常有用，另外当您在一个大型的团队中工作或在一个大型的代码库中，同时将所有的类放在您的头脑中是困难或不可能的时，它也可以帮助您。

### 2.abc怎么工作？

abc通过把基类中的方法标记为抽象方法，并且注册具体类为基类的实现的方式工作。

### ABCMeta
用于定义抽象基类（ABC）的元类。

元类就是用来创建这些类（对象）的类，元类就是类的类

简单的说ABCMeta就是让你的类变成一个纯虚类，子类必须实现某个方法，这个方法在父类中用@abc.abstractmethod修饰

在纯虚类中你可以实现abstractmethod修饰的虚方法，也可以不实现
不实现的话，子类中就必须实现abstractmethod修饰的函数，否则就会报错


In [ ]:
# abc：Abstract Base Classes
# 作用：在代码中定义和使用抽象基类进行API检查。
from abc import ABCMeta,abstractmethod
from multiprocessing import Pool
from functools import partial
import numpy as np

class loss(metaclass=ABCMeta):
  '''
  The abstract base class for loss function.
  Three things should be specified for a loss,
  namely link function(链接函数), gradient(梯度) and hessian(海森矩阵).
  link() is the link function, which take scores as input, and returns predictions.
  g() is the gradient, which takes true values and scores as input, and returns gradient.
  h() is the hessian, which takes true values and scores as input, and returns hessian.
  All inputs and outputs are numpy array.
  '''
  @abstactmethod
  def link(self,score):
    pass
  
  @abstactmethod
  def g(self,true,score):
    pass
  
  @abstractmethod
  def h(self,true,score):
    pass

# gbdt 处理回归问题
class mse(loss):
  '''Loss class for mse. As for mse, link function is pred=score'''
  def link(self,score):
    return score
  
  def g(self,true,score):
    return score-true  # ?
  
  def h(self,true,score):
    return np.ones_like(score) # 返回一个用1填充的跟输入 形状和类型 一致的数组

# gbdt处理 二分类问题
class log(loss):
  '''Loss class for log loss. As for log loss, link function is logistic transformation.'''
  def link(self,score):
    return 1/(1 + np.exp(-score)) # 1/(1 + np.exp(-f(x))) 
  
  def g(self,score,true):
    pred = self.link(score):
    return pred - true #  梯度，即对 gbdt二分类问题对f(x) 进行求导
  
  def h(self,true,score):
    pred = self.link(-score)
    return pred * (1 - pred) # 对 gbdt的损失函数进行二阶求导，即对上面的 pred-true 对f(x)进行再次求导,true 的导数为 0，即为逻辑回归的导数

class GBDT(object):
  '''
  根据代码来看是 XGBoost 代码
  Parameters:
  ----------
  n_threads: The number of threads used for fitting and predicting.
  loss: Loss function for gradient boosting.
        'mse' for regression task and 'log' for classfication task.
        A child class of the loss class could be passed to implement customized loss.
  
  max_depth: The maximum depth of a tree.
  min_sample_split: The minimum number of samples required to further split a node.
  reg_lamda: The regularization coefficient for leaf score, also known as lambda.
  gamma: The regularization coefficient for number of tree nodes, also known as gamma.
  learning_rate: The learning rate of gradient boosting.
  n_estimators: Number of trees.
  '''
  def __init__(self
               n_threads=None,
               lose='mse',
               max_depth=3,min_sample_split=10,reg_lambda=1,gamma=0,
               learning_rate=0.1,n_estimators=100):
    self.n_threads=n_threads
    self.loss=loss
    self.max_depth=max_depth
    self.min_sample_split=min_sample_split
    self.reg_lambda=re_lambda
    self.gamma=gamma
    self.learning_rate=learning_rate
    self.n_estimators=n_estimators
  
  def fit(self,train,target):
    self.estimators=[]
    if self.loss=='mse':
      self.loss=mse()
    if self.loss=='log':
      self.loss=log()
    self.score_start = target.mean()
    score=np.ones(len(train))*self.score_start
    for i in range(self.n_estimators):
      estimator=Tree(n_threads=self.n_threads,
                     max_depth=self.max_depth,min_sample_split=self.min_sample_split,reg_lambda=self.reg_lambda,gamma=self.gamma)
      estimator.fit(train,g=self.loss.g(target,score),h=self.loss.h(target,score))
      self.estimators.append(estimator)
      score+=self.learning_rate*estimator.predict(train)
    return self
  
  def predict(self,test):
    score=np.ones(len(test))*self.score_start
    for i in range(self.n_estimators):
      score+=self.learning_rate*self.estimators[i].predict(test)
    return self.loss.link(score)
  

class TreeNode(object):
  '''
  The data structure that are used for storing trees.
  A tree is presented by a set of nested TreeNodes,
  with one TreeNode pointing two TreeNodes,
  until a tree leaf is reached.

  Parameters:
  -----------
  is_leaf: If is TreeNode is a lead.
  score: The prediction (score) of a tree leaf.
  split_feature: The split feature of a tree node.
  split_threshold: The split threshold of a tree node.
  left_child: Pointing to a child TreeNode,
          where the value of split feature is less than the  split  threshold.
  right_child:  Pointing a child TreeNode,
          where the value of split feature is greater than or equal to the split threshold.
  '''

  def __init__(self,
               is_leaf=False,
               score=None,
               split_feature=None,
               split_threshold=None,
               left_child=None,right_child=None):
    self.is_leaf=is_leaf
    self.score=score
    self.split_feature=split_feature
    self.split_threshold=split_threshold
    self.left_child=left_child
    self.right_child=right_child


class Tree(object):
  '''
  This is the building block for GBDT,
  which is a single decision tree,
  also known as an estimator.

  Parameters:
  ----------
  n_threads: The number of threads used for fitting and predicting.
  max_depth: The maximum depth of the tree.
  min_sample_split: The minimum number of samples required to further split a node.
  reg_lambda: The regularization coefficient for leaf prediction(叶结点预测的正则化系数), also known as lambda.
  gamma: The regularization coefficient for number of TreeNode(TreeNode个数的正则化系数), also known as gamma.
  '''
  def __init__(self,n_threads=None,
               max_depth=3,
               min_sample_split=10,
               reg_lambda=1,gamma=0):
    self.n_threads=n_threads
    self.max_depth=max_depth
    self.min_sample_split=min_sample_split
    self.reg_lambda=reg_lambda
    self.gamma=gamma

  def fit(self,train,g,h):
    '''
    All inputs must be numpy arrays.
    g and h are gradient and hessian repectively.
    '''
    self.estimator=self.construct_tree(train,g,h,self.max_depth)
    return self
  
  def predict(self,test):
    '''
    test must be numpy array.
    Return predictions (scores) as an array.
    Multiprocessing is supported for prediction.
    '''
    pool=Pool(self.n_threads)
    f=partial(self.predict_single,self.estimator)
    result=np.array(pool.map(f,test))
    pool.close() #关闭pool，使其不在接受新的（主进程）任务
    pool.join() # 主进程阻塞后，等待子进程的退出， join方法要在close或terminate之后使用。主进程阻塞后，让子进程继续运行完成，子进程运行完后，再把主进程全部关掉。
    return result
  
  
  def predict_single(self,treenode,test):
    '''
    The predict method for a single sample point.
    test must be numpy array.
    Return prediction (score) as a number.
    '''
    if treenode.is_leaf:
      return treenode.score
    else:
      if test[treenode.split_feature] < treenode.split_threshold:
        return self.predict_single(treenode.left_child,test)
      else:
        return self.predict_single(treenode.right_child,test)
  
  def construct_tree(self,train,g,h,max_depth):
    '''
    Construct tree recursively.
    First we should check if we should stop further splitting.
    The stopping conditions include:
    1. We have reached the pre-determined max_depth
    2. The number of sample points at this node is less than min_sample_split
    3. The best gain is less than gamma.
    4. Targets take only onee value.
    5. Each feature takes only one value.
    Be careful design, we could avoid checking condition 4 and 5 explicitly(明确地).
    In function find_threshold(), the best_gain is sets to 0 initially.
    So if there are no further feature to split,
    or all the targets take the same value,
    the return value of best_gain would zero.
    Thus condition 3 would be satisfied,
    and no further splitting would be done.
    To conclude, we need only to check condition 1,2 and 3.
    '''
    if max_depth==0 or len(train)<self.min_sample_split:
      return TreeNode(is_leaf=True,score=self.leaf_score(g,h))
    
    feature,threshold,gain = self.find_best_split(train,g,h)
    if gain <= self.gamma:
      return TreeNode(is_leaf=True,score=self.leaf_score(g,h))
    
    index=train[:,feature]<threshold
    left_child=self.contruct_tree(train[index],g[index],h[index],max_depth-1)
    right_child=self.construct_tree(train[~index],g[~index],h[~index],depth-1)
    return TreeNode(split_feature=feature,split_threshold=threshold,left_child=left_child,right_child=right_child)
  
  def leaf_score(self,g,h):
    '''
    Given the gradient and hessian of a tree leaf,
    return the prediction (score) at this leaf.
    The score is -G/(H+λ)
    叶子结点的权重
    '''
    return -np.sum(g)/(np.sum(h) + self.reg_lambda)
  
  def leaf_loss(self,g,h):
    '''
    Given the gradient and hessian of a tree leaf,
    return the minimized loss at this leaf.
    The minimized loss is -0.5*G^2/(H+λ).
    '''
    return -0.5*np.square(np.sum(g))/(np.sum(h)+self.reg_lambda)
  
  def find_best_split(self,train,g,h):
    '''
    Return the best feature to split together with the corresponding threshold.
    (返回拆分的最佳特征以及相应的阈值)
    Each feature is scanned by find_threshold(),
    a [threshold,best_gain] list is returned for each feature.
    Then we select the feature with the largest best_gain,
    and return index of that feature, the threshold, and the gain that is achieved.
    '''
    pool=Pool(self.n_threads)
    f=partial(self.find_threshold,g,h)
    thresholds=np.array(pool.map(f,train.T))
    pool.close()
    pool.join()
    feature=np.argmax(thresholds[:,1],axis=0)
    threshold=thresholds[feature,0]
    gain=thresholds[feature,1]
    return feature,threshold,gain

  def find_threshold(self,g,h,train):
    '''
    Given a particular feature,
    return the best split threshold together with the gain that is achieved.
    '''
    loss=self.leaf_loss(g,h)
    threshold=None
    best_gain=0
    unq=np.unique(train)
    for i in range(1,len(unq)):
      this_threshold=(unq[i-1]+unq[i])/2
      index=train<this_threshold
      left_loss=self.leaf_loss(g[index],h[index])
      right_loss=self.leaf_loss(g[~index],h[~index])
      this_gain=loss - left_loss - right_loss
      if this_gain > best_gain:
        threshold=this_threshold
        best_gain=this_gain
    return [threshold,best_gain]







    

  

  
